<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/ai-powered-search/14_3_qa_model_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In this notebook, we"re going to install a transformer model, analyze the embedding output, and compare some vectors

In [ ]:
#outdoors
![ ! -d 'outdoors' ] && git clone --depth=1 https://github.com/ai-powered-search/outdoors.git
! cd outdoors && git pull
! cd outdoors && cat outdoors.tgz.part* > outdoors.tgz
! cd outdoors && mkdir -p '../data/outdoors/' && tar -xvf outdoors.tgz -C '../data/outdoors/'

Cloning into 'outdoors'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 0), reused 22 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (25/25), 491.39 MiB | 23.89 MiB/s, done.
Updating files: 100% (23/23), done.
Already up to date.
README.md
concepts.pickle
._guesses.csv
guesses.csv
._guesses_all.json
guesses_all.json
outdoors_concepts.pickle
outdoors_embeddings.pickle
._outdoors_golden_answers.csv
outdoors_golden_answers.csv
._outdoors_golden_answers.xlsx
outdoors_golden_answers.xlsx
._outdoors_golden_answers_20210130.csv
outdoors_golden_answers_20210130.csv
outdoors_labels.pickle
outdoors_question_answering_contexts.json
outdoors_questionanswering_test_set.json
outdoors_questionanswering_train_set.json
._posts.csv
posts.csv
predicates.pickle
pull_aips_dependency.py
._question-answer-seed-contexts.csv
question-answer-seed-contexts.csv
question-answer-squad2-guesses

In [ ]:
%%capture

!pip install datasets

In [2]:
import sys
import os
import time
sys.path.append("../..")
import pandas as pd
import numpy as np
import pickle
import json
import tqdm
import random

import torch

from transformers import pipeline
from datasets import Dataset, DatasetDict


from IPython.display import display,HTML

In [3]:
def get_processor_type():
    gpu_device = torch.device("cuda:0")
    cpu_device = torch.device("cpu")
    return gpu_device or cpu_device

def get_processor_device():
    return 0 if torch.cuda.is_available() else -1

In [4]:
print("Processor: " + str(get_processor_type()))
print("Device id: " + str(get_processor_device()))

Processor: cuda:0
Device id: 0


In [ ]:
# loading a question-answering dataset
contexts = pd.read_csv("data/outdoors/question-answer-seed-contexts.csv")
contexts.head()

,id,question,context,url
0,4410,Who places the anchors that rock climbers use?,There are two distinct styles of free rock cli...,https://outdoors.stackexchange.com/questions/4410
1,5347,"Who places the bolts on rock climbing routes, ...",What you're talking about is Sport climbing. G...,https://outdoors.stackexchange.com/questions/5347
2,20662,Who gets the bill if you activate a PLB to hel...,"Almost always the victim gets the bill, but as...",https://outdoors.stackexchange.com/questions/2...
3,7623,What knot is this one? What are its purposes?,Slip knot It's undoubtably a slip knot that's ...,https://outdoors.stackexchange.com/questions/7623
4,11587,"What sort of crane, and what sort of snake?","To answer the snake part of it, looking at som...",https://outdoors.stackexchange.com/questions/1...


## Creating silver dataset

In [ ]:
def get_processor_device():
    return 0 if torch.cuda.is_available() else -1

def display_guesses(guesses):
    display(HTML(pd.DataFrame(guesses[0:10]).to_html(index=False)))

In [ ]:
model_name = "deepset/roberta-base-squad2"
device = get_processor_device()

In [ ]:
def answer_questions(contexts, k=10):
  nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=device)
  guesses = []
  for _, row in tqdm.tqdm(contexts[0:k].iterrows(), total=k):
    results = nlp({
        'context': row['context'],
        'question': row['question']
    })
    guesses.append(results)
  return guesses

In [ ]:
# Automatically labeling data from a pretrained model
guesses = answer_questions(contexts, k=len(contexts))

In [ ]:
display_guesses(guesses)

score,start,end,answer
0.278927,474,516,a local enthusiast or group of enthusiasts
0.200848,81,117,the person who is creating the climb
0.018632,14,24,the victim
0.222317,29,38,slip knot
0.000551,1255,1262,aquatic
0.374997,15,40,a high-tech treasure hunt
0.563755,192,232,"a tube of lightweight, stretchy material"
0.110915,125,154,the cheapest one of the three
0.805174,68,76,blocking
0.247008,227,265,the traditional longbow made from wood


In [ ]:
contexts["answers"] = guesses
contexts.to_csv("data/outdoors/qa-squad2-guesses.csv", index=False)

##Human-in-the-loop training

In [ ]:
random.seed(0)

def get_training_data(filename):
    golden_answers = pd.read_csv(filename)
    golden_answers = golden_answers[golden_answers["class"] != None]
    qa_data = []
    for _, row in golden_answers.iterrows():
        answers = row["gold"].split("|")
        starts = [row["context"].find(a) for a in answers]
        missing = -1 in starts
        if not missing:
            row["title"] = row["question"]
            row["answers"] = {"text": answers, "answer_start": starts}
            qa_data.append(row)
    columns = ["id", "url", "title", "question", "context", "answers"]
    df = pd.DataFrame(qa_data, columns=columns).sample(frac=1)
    train_split = int(len(df) * 0.75)
    eval_split = (int((len(df) - train_split) / 1.25) +
                  train_split - 1)
    train_dataset = Dataset.from_pandas(df[:train_split])
    test_dataset = Dataset.from_pandas(df[train_split:eval_split])
    validation_dataset = Dataset.from_pandas(df[eval_split:])
    return DatasetDict({"train": train_dataset, "test": test_dataset,"validation": validation_dataset})

In [ ]:
#It took about 2-3 hours to label 200 question/answer rows
#Doing so will give you a deeper appreciation for the difficulty of the NLP task.
#I *highly* encourage you to label even more documents, and re-run the fine-tuning tasks coming up.
datadict = get_training_data("data/outdoors/outdoors_golden_answers.csv")
model_path = "data/question-answering/question-answering-training-set"

#datadict.save_to_disk(model_path)

datadict

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'question', 'context', 'answers', '__index_level_0__'],
        num_rows: 1243
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'question', 'context', 'answers', '__index_level_0__'],
        num_rows: 331
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'question', 'context', 'answers', '__index_level_0__'],
        num_rows: 84
    })
})